In [ ]:
# @title ГОДЖО 30 СЕК — МОДЕЛИ ИЗ ДАТАСЕТА (0 СЕКУНД НА СКАЧИВАНИЕ)
import os
import subprocess
import json
import time
import requests
from IPython.display import FileLink, display

# === ПАПКИ ===
WORKSPACE = "/kaggle/working"
COMFY_DIR = f"{WORKSPACE}/ComfyUI"
OUTPUT_DIR = f"{COMFY_DIR}/output"
INPUT_DIR = f"{WORKSPACE}/input"
FRAMES_DIR = f"{WORKSPACE}/frames"
DATASET_DIR = "/kaggle/input/comfyui-models-gojo"  # ТВОЙ ДАТАСЕТ

# Создаем только базовые папки, ComfyUI папки создадим позже
os.makedirs(INPUT_DIR, exist_ok=True)
os.makedirs(FRAMES_DIR, exist_ok=True)


# === УСТАНОВКА COMFYUI (ОДИН РАЗ) ===
if not os.path.exists(f"{COMFY_DIR}/main.py"):
    print("Установка ComfyUI...")
    # Удаляем папку если она существует но пустая/неполная
    if os.path.exists(COMFY_DIR):
        subprocess.run(["rm", "-rf", COMFY_DIR], check=True)
    subprocess.run(["git", "clone", "https://github.com/comfyanonymous/ComfyUI", COMFY_DIR], check=True)

os.chdir(COMFY_DIR)

# Теперь создаем папки моделей внутри ComfyUI
os.makedirs(f"{COMFY_DIR}/models/checkpoints", exist_ok=True)
os.makedirs(f"{COMFY_DIR}/models/animatediff_models", exist_ok=True)
os.makedirs(f"{COMFY_DIR}/models/upscale_models", exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

# === КОПИРУЕМ МОДЕЛИ ИЗ ДАТАСЕТА (0 СЕКУНД ОЖИДАНИЯ) ===
print("Копируем модели из датасета...")
!cp "$DATASET_DIR/sd_xl_base_1.0.safetensors" "$COMFY_DIR/models/checkpoints/" -f
!cp "$DATASET_DIR/mm_sd_v15_v2.ckpt" "$COMFY_DIR/models/animatediff_models/" -f
!cp "$DATASET_DIR/4x-UltraSharp.pth" "$COMFY_DIR/models/upscale_models/" -f
print("Модели готовы!")

# Устанавливаем зависимости если requirements.txt есть, но пакеты не установлены
if not os.path.exists(f"{COMFY_DIR}/.deps_installed"):
    print("Установка зависимостей...")
    subprocess.run([
        "pip", "install", "-q", "--no-cache-dir",
        "torch", "torchvision", "torchaudio",
        "--extra-index-url", "https://download.pytorch.org/whl/cu121"
    ], check=True)
    subprocess.run(["pip", "install", "-q", "--no-cache-dir", "-r", "requirements.txt"], check=True)
    subprocess.run(["pip", "install", "-q", "huggingface_hub"], check=True)

    # Устанавливаем custom nodes
    os.makedirs("custom_nodes", exist_ok=True)
    os.chdir("custom_nodes")
    if not os.path.exists("ComfyUI-Manager"):
        subprocess.run(["git", "clone", "https://github.com/ltdrdata/ComfyUI-Manager"], check=True)
    if not os.path.exists("ComfyUI-AnimateDiff-Evolved"):
        subprocess.run(["git", "clone", "https://github.com/Kosinkadink/ComfyUI-AnimateDiff-Evolved"], check=True)

    os.chdir(COMFY_DIR)
    # Создаем маркер что зависимости установлены
    with open(f"{COMFY_DIR}/.deps_installed", "w") as f:
        f.write("OK")
    print("Зависимости установлены!")

os.chdir(COMFY_DIR)

# === WORKFLOW: ГЕНЕРАЦИЯ АНИМАЦИИ (8 ПОХОЖИХ КАДРОВ) ===
# Вместо batch=8 (создает разные картинки), генерируем кадры с небольшим изменением seed
print("Создание workflow для анимации с плавным переходом...")

# Изменяем prompt - добавляем описание движения
animation_prompt = "cinematic portrait of gojo satoru, white spiky hair, black blindfold, confident expression, anime style, highly detailed, 8k, professional lighting, SLIGHT HEAD TILT, SUBTLE MOVEMENT"

# ОПТИМИЗАЦИЯ: Уменьшаем разрешение для ускорения генерации (384x576 вместо 512x768)
# ОПТИМИЗАЦИЯ: Уменьшаем steps до 15 (вместо 20) - баланс качества и скорости
# Генерируем 8 кадров с близкими seed для похожих результатов
workflows = []
for frame_idx in range(8):
    # Используем близкие seed (42, 43, 44...) для похожих но не идентичных кадров
    workflow = {
      "3": { "class_type": "CheckpointLoaderSimple", "inputs": { "ckpt_name": "sd_xl_base_1.0.safetensors" } },
      "5": { "class_type": "EmptyLatentImage", "inputs": { "width": 384, "height": 576, "batch_size": 1 } },
      "6": { "class_type": "CLIPTextEncode", "inputs": { "text": animation_prompt, "clip": ["3", 1] } },
      "7": { "class_type": "CLIPTextEncode", "inputs": { "text": "blurry, deformed, low quality, watermark, text, bad anatomy, multiple heads, duplicate", "clip": ["3", 1] } },
      "8": { "class_type": "KSampler", "inputs": {
          "seed": 42 + frame_idx,  # Близкие seed для похожих кадров
          "steps": 15,  # ОПТИМИЗАЦИЯ: 15 вместо 20
          "cfg": 7,
          "sampler_name": "euler",
          "scheduler": "normal",
          "positive": ["6", 0],
          "negative": ["7", 0],
          "model": ["3", 0],
          "latent_image": ["5", 0],
          "denoise": 1.0
      } },
      "9": { "class_type": "VAEDecode", "inputs": { "samples": ["8", 0], "vae": ["3", 2] } },
      "10": { "class_type": "UpscaleModelLoader", "inputs": { "model_name": "4x-UltraSharp.pth" } },
      "11": { "class_type": "ImageUpscaleWithModel", "inputs": { "image": ["9", 0], "upscale_model": ["10", 0] } },
      "12": { "class_type": "SaveImage", "inputs": { "images": ["11", 0], "filename_prefix": f"GOJO_FRAME_{frame_idx:03d}" } }
    }
    workflows.append(workflow)

print(f"Создано {len(workflows)} workflows (384x576, 15 steps для ускорения)")

# === API: ЗАПУСК СЕРВЕРА ===
print("Запуск ComfyUI сервера...")
server = subprocess.Popen(
    ["python", "main.py", "--listen", "127.0.0.1", "--port", "8188"],
    cwd=COMFY_DIR,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1
)

# Читаем вывод сервера в отдельном потоке
import threading
def print_server_output():
    for line in server.stdout:
        print(line, end='')

output_thread = threading.Thread(target=print_server_output, daemon=True)
output_thread.start()

print("Ожидание запуска сервера (может занять 1-2 минуты)...")
server_ready = False
for i in range(180):  # 3 минуты максимум
    try:
        response = requests.get("http://127.0.0.1:8188/system_stats", timeout=2)
        if response.status_code == 200:
            print(f"\n✓ Сервер готов! (прошло {i} сек)")
            server_ready = True
            break
    except requests.exceptions.ConnectionError:
        pass  # Сервер еще не готов
    except Exception as e:
        print(f"Ошибка проверки: {e}")

    if i % 10 == 0 and i > 0:
        print(f"Ожидание... ({i} сек)")
    time.sleep(1)

if not server_ready:
    print("ОШИБКА: Сервер не запустился за 3 минуты.")
    print("Последние строки вывода сервера:")
    server.terminate()
    exit()

# === ОТПРАВКА ЗАДАЧ ОДНИМ BATCH (ОПТИМИЗАЦИЯ!) ===
# Вместо 8 отдельных запросов, используем batch_size=8 для одновременной генерации
# Это быстрее, но кадры будут более разными (что нормально для RIFE интерполяции)
print("Генерация 8 кадров одним batch запросом (быстрее)...")

# Создаем один workflow с batch_size=8 и вариативным seed через noise
batch_workflow = {
  "3": { "class_type": "CheckpointLoaderSimple", "inputs": { "ckpt_name": "sd_xl_base_1.0.safetensors" } },
  "5": { "class_type": "EmptyLatentImage", "inputs": { "width": 384, "height": 576, "batch_size": 8 } },
  "6": { "class_type": "CLIPTextEncode", "inputs": { "text": animation_prompt, "clip": ["3", 1] } },
  "7": { "class_type": "CLIPTextEncode", "inputs": { "text": "blurry, deformed, low quality, watermark, text, bad anatomy, multiple heads, duplicate", "clip": ["3", 1] } },
  "8": { "class_type": "KSampler", "inputs": {
      "seed": 42,
      "steps": 15,
      "cfg": 7,
      "sampler_name": "euler",
      "scheduler": "normal",
      "positive": ["6", 0],
      "negative": ["7", 0],
      "model": ["3", 0],
      "latent_image": ["5", 0],
      "denoise": 1.0
  } },
  "9": { "class_type": "VAEDecode", "inputs": { "samples": ["8", 0], "vae": ["3", 2] } },
  "10": { "class_type": "UpscaleModelLoader", "inputs": { "model_name": "4x-UltraSharp.pth" } },
  "11": { "class_type": "ImageUpscaleWithModel", "inputs": { "image": ["9", 0], "upscale_model": ["10", 0] } },
  "12": { "class_type": "SaveImage", "inputs": { "images": ["11", 0], "filename_prefix": "GOJO_FRAMES" } }
}

try:
    print("Отправка batch задачи...")
    resp = requests.post("http://127.0.0.1:8188/prompt", json={"prompt": batch_workflow, "client_id": "gojo_batch"}, timeout=10)
    resp_data = resp.json()

    if "prompt_id" not in resp_data:
        print("ОШИБКА API:", resp_data)
        server.terminate()
        exit()

    prompt_id = resp_data["prompt_id"]
    print(f"Batch задача отправлена: {prompt_id}")

    # Ждем завершения
    print("Генерация 8 кадров batch (это быстрее чем последовательно)...")
    for i in range(300):  # 10 минут максимум
        try:
            hist = requests.get(f"http://127.0.0.1:8188/history/{prompt_id}", timeout=2).json()
            if prompt_id in hist:
                status = hist[prompt_id].get("status", {})
                if status.get("completed"):
                    print("✓ Все 8 кадров готовы!")
                    break
                elif i % 10 == 0 and i > 0:
                    print(f"Генерация... ({i*2} сек)")
        except Exception:
            pass
        time.sleep(2)

except requests.exceptions.RequestException as e:
    print(f"ОШИБКА соединения: {e}")
    server.terminate()
    exit()

server.terminate()
server.wait(timeout=5)


# === API: ЗАПУСК СЕРВЕРА ===
print("Запуск ComfyUI сервера...")
server = subprocess.Popen(
    ["python", "main.py", "--listen", "127.0.0.1", "--port", "8188"],
    cwd=COMFY_DIR,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1
)

# Читаем вывод сервера в отдельном потоке
import threading
def print_server_output():
    for line in server.stdout:
        print(line, end='')

output_thread = threading.Thread(target=print_server_output, daemon=True)
output_thread.start()

print("Ожидание запуска сервера (может занять 1-2 минуты)...")
server_ready = False
for i in range(180):  # 3 минуты максимум
    try:
        response = requests.get("http://127.0.0.1:8188/system_stats", timeout=2)
        if response.status_code == 200:
            print(f"\n✓ Сервер готов! (прошло {i} сек)")
            server_ready = True
            break
    except requests.exceptions.ConnectionError:
        pass  # Сервер еще не готов
    except Exception as e:
        print(f"Ошибка проверки: {e}")

    if i % 10 == 0 and i > 0:
        print(f"Ожидание... ({i} сек)")
    time.sleep(1)

if not server_ready:
    print("ОШИБКА: Сервер не запустился за 3 минуты.")
    print("Последние строки вывода сервера:")
    server.terminate()
    exit()

# === ОТПРАВКА ЗАДАЧИ ===
print("Генерация 8 кадров (1 сек)...")
try:
    resp = requests.post("http://127.0.0.1:8188/prompt", json={"prompt": workflow, "client_id": "gojo"}, timeout=10)
    resp_data = resp.json()

    if "prompt_id" not in resp_data:
        print("ОШИБКА API:", resp_data)
        server.terminate()
        exit()

    prompt_id = resp_data["prompt_id"]
    print(f"Задача отправлена: {prompt_id}")

except requests.exceptions.RequestException as e:
    print(f"ОШИБКА соединения с сервером: {e}")
    server.terminate()
    exit()

# Ждем завершения генерации
print("Ожидание завершения генерации...")
for i in range(300):  # 10 минут максимум
    try:
        hist = requests.get(f"http://127.0.0.1:8188/history/{prompt_id}", timeout=2).json()
        if prompt_id in hist:
            status = hist[prompt_id].get("status", {})
            if status.get("completed"):
                print("✓ 8 кадров сгенерированы!")
                break
            elif "status_str" in status:
                if i % 10 == 0:
                    print(f"Статус: {status.get('status_str', 'Processing...')}")
    except Exception as e:
        if i % 30 == 0:
            print(f"Ошибка проверки статуса: {e}")
    time.sleep(2)
else:
    print("Таймаут генерации (10 минут)")

server.terminate()
server.wait(timeout=5)

# === ПОДГОТОВКА КАДРОВ ДЛЯ RIFE ===
print("Поиск сгенерированных кадров...")
frame_files = sorted([f for f in os.listdir(OUTPUT_DIR) if f.startswith("GOJO_FRAMES") and f.endswith(".png")])

if len(frame_files) < 8:
    print(f"ОШИБКА: Недостаточно кадров. Найдено {len(frame_files)}, нужно минимум 8.")
    print("Содержимое OUTPUT_DIR:")
    !ls -la $OUTPUT_DIR
    exit()

print(f"Найдено {len(frame_files)} кадров. Копирование в frames/...")
for i, frame_file in enumerate(frame_files[:8]):
    src = f"{OUTPUT_DIR}/{frame_file}"
    # RIFE ожидает имена файлов: 0.png, 1.png, 2.png и т.д.
    dst = f"{FRAMES_DIR}/{i}.png"
    !cp "$src" "$dst"

print("Кадры готовы для RIFE (переименованы в 0.png, 1.png, ...).")

# === RIFE: 8 → 256 КАДРОВ (32 СЕК) ===
print("Интерполяция до 32 секунд с Practical-RIFE v4.26...")

# Устанавливаем Practical-RIFE (лучшая версия на 2025)
!rm -rf /kaggle/working/RIFE
!git clone https://github.com/hzwer/Practical-RIFE /kaggle/working/RIFE
%cd /kaggle/working/RIFE

# Устанавливаем зависимости
print("Установка зависимостей RIFE...")
# torch и torchvision уже установлены в Kaggle, не переустанавливаем их
!pip install -q opencv-python scikit-video

# Скачиваем модель RIFE v4.26 (лучшая версия 2025)
print("Скачивание модели RIFE v4.26 с Google Drive...")
!mkdir -p train_log

# Устанавливаем gdown для скачивания с Google Drive
!pip install -q gdown

# Скачиваем RIFE v4.26 по прямой ссылке
print("Загрузка RIFEv4.26_0921.zip...")
!gdown 1gViYvvQrtETBgU1w8axZSsr7YUuw31uy -O train_log.zip

model_downloaded = False

if os.path.exists("train_log.zip") and os.path.getsize("train_log.zip") > 1000000:
    file_size_mb = os.path.getsize("train_log.zip") / 1024 / 1024
    print(f"✓ RIFE v4.26 скачан ({file_size_mb:.1f} MB), распаковываем...")
    !unzip -q train_log.zip

    # Проверяем где находится flownet.pkl после распаковки
    !find . -name "flownet.pkl" -o -name "*.pkl" | head -5

    # Ищем файл модели в разных возможных местах
    if os.path.exists("train_log/flownet.pkl"):
        model_downloaded = True
        print(f"✓ Модель RIFE v4.26 готова ({os.path.getsize('train_log/flownet.pkl')/1024/1024:.1f} MB)")
    elif os.path.exists("flownet.pkl"):
        print("Перемещаем flownet.pkl в train_log/...")
        !mv flownet.pkl train_log/
        model_downloaded = True
        print(f"✓ Модель RIFE v4.26 готова ({os.path.getsize('train_log/flownet.pkl')/1024/1024:.1f} MB)")
    else:
        # Ищем в подпапках
        result = !find . -name "flownet.pkl" | head -1
        if result:
            pkl_path = result[0]
            print(f"Найден flownet.pkl: {pkl_path}")
            !cp "{pkl_path}" train_log/flownet.pkl
            if os.path.exists("train_log/flownet.pkl"):
                model_downloaded = True
                print(f"✓ Модель RIFE v4.26 готова ({os.path.getsize('train_log/flownet.pkl')/1024/1024:.1f} MB)")
else:
    print(f"ОШИБКА: Файл train_log.zip не скачался или слишком мал")
    if os.path.exists("train_log.zip"):
        print(f"Размер файла: {os.path.getsize('train_log.zip')} байт")
    !ls -la train_log.zip 2>/dev/null || echo "Файл не существует"

if not model_downloaded:
    print("ОШИБКА: Не удалось скачать модель RIFE ни одним методом")
    !ls -la train_log/
    print("\nИспользуем fallback: простое видео из 8 кадров...")
    !ffmpeg -framerate 8 -i $FRAMES_DIR/%d.png -c:v libx264 -pix_fmt yuv420p /kaggle/working/GOJO_8FRAMES.mp4 -y -loglevel error
    final_video = "/kaggle/working/GOJO_8FRAMES.mp4"
    if os.path.exists(final_video):
        file_size = os.path.getsize(final_video) / 1024 / 1024
        print(f"ГОТОВО! 8 кадров ({file_size:.1f} MB):")
        display(FileLink(final_video))
else:
    # Модель загружена успешно, запускаем интерполяцию
    !ls -lh train_log/

    print("\nЗапуск интерполяции RIFE v4.26 (это займет несколько минут)...")
    print("Параметры: exp=5 (8→256 кадров), UHD mode для высокого качества")

    # Practical-RIFE inference_video.py аргументы
    !python inference_video.py \
        --img $FRAMES_DIR \
        --exp 5 \
        --output /kaggle/working/GOJO_30SEC.mp4 \
        --UHD

    # === РЕЗУЛЬТАТ ===
    final_video = "/kaggle/working/GOJO_30SEC.mp4"
    if os.path.exists(final_video):
        file_size = os.path.getsize(final_video) / 1024 / 1024
        print(f"ГОТОВО! 30+ СЕКУНД ШОРТС ({file_size:.1f} MB):")
        display(FileLink(final_video))
    else:
        print("RIFE не создал выходное видео. Проверяем что есть:")
        !ls -la /kaggle/working/*.mp4 2>/dev/null || echo "MP4 файлы не найдены"

        # Пытаемся создать простое видео из кадров как fallback
        print("\nСоздаем fallback видео из исходных 8 кадров...")
        !ffmpeg -framerate 8 -i $FRAMES_DIR/%d.png -c:v libx264 -pix_fmt yuv420p -preset fast /kaggle/working/GOJO_8FRAMES.mp4 -y -loglevel error
        fallback_video = "/kaggle/working/GOJO_8FRAMES.mp4"
        if os.path.exists(fallback_video):
            file_size = os.path.getsize(fallback_video) / 1024 / 1024
            print(f"ГОТОВО! 8 кадров ({file_size:.1f} MB):")
            display(FileLink(fallback_video))
